In [7]:
import pandas as pd
import nltk
import re
import numpy as np
import os
import string
import emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS #These are Dict

In [8]:
#Use this to extend the viewing width of Jupyter
pd.set_option('display.max_colwidth', -1)

C:\ProgramData\Anaconda3\envs\TensorFlowGPU\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [9]:
df = pd.read_csv('mbti_1.csv')
df["posts"]= df['posts'].astype(str)

In [10]:
#Removing web address, ..., numbers, and delimiters(|||)
def cleaner(comments):
    #comments = re.sub('http\S+ ','',comments)#removing web addresses
    comments = re.sub('\w+:\/\/\S+','',comments)#removing web addresses
    comments = re.sub('\.\.\.','', comments)#removing triple...
    comments = re.sub('[0-9]+','', comments)#removing numbers
    comments = re.sub('\|\|\|', '', comments)# removing delimiters
    for emote, mean in EMOTICONS.items():
        comments = comments.replace(emote, ' <'+mean+'> ')
    comments = re.sub(r':(\w+):', r' <\1> ', comments) # we are capturing group1 :group1: and returning <group1>
    return comments
df['Clean']= df['posts'].apply(lambda x: cleaner(x))

In [11]:
"""
Uncomment if you neeed to Clean_v1.csv 
"""
#df.to_csv("Clean_v1.csv")

In [12]:
#Manually Replacing Emojis that are not compatible to the EMOTICONS 
def emoji(comments):
    comments = re.sub('\@\.\@', ' <confused> ',comments)
    comments=re.sub('\:D', ' <happy> ', comments)
    comments=re.sub('\>\.\>',' <opsies> ',comments)
    comments=re.sub('\<\.\<',' <opsies> ',comments)
    comments=re.sub('\;D',' <winky> ',comments)
    comments=re.sub('\:\)',' <happy> ',comments)
    comments= re.sub("\>\.\<", ' <opsies> ', comments)
    comments= re.sub("\>\<", ' <opsies> ', comments)
    comments= re.sub("\)\:", ' <sad> ', comments)
    comments= re.sub("xD", ' <laughing> ', comments)
    comments= re.sub("\:P", ' <naughty> ', comments)
    comments= re.sub("\;\)", ' <winky> ', comments)
    return comments
df['Clean']=df['Clean'].apply(lambda x: emoji(x))

In [13]:
#Removing Stop Words NLTK
from nltk.corpus import stopwords
stopwordlist = stopwords.words('english')

def stopWord(comments):
    lst = []
    wordList =nltk.word_tokenize(comments)
    #wordList = comments.split()
    for word in wordList:
        if word.lower() not in stopwordlist:
            lst.append(word.lower())
    return lst
df['StopWord']= df['Clean'].apply(lambda x: stopWord(x))

In [14]:
#Lemmatize/Stemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def lemmatize_words(posts):
    lemmatizer = WordNetLemmatizer()
    # lemmatization process depends on the POS tag to come up with the correct lemma, so we set up word net map
    stem_words = []
    wordnet_map = {"N": wordnet.NOUN, "V": wordnet.VERB, "J": wordnet.ADJ, "R": wordnet.ADV}
    tagged = nltk.pos_tag(posts)
    stemmed = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in tagged]
    return stemmed
df['Lemmatize']= df['StopWord'].apply(lambda x: lemmatize_words(x))

In [15]:
#Remove Punctuation
symbollist = '''[\?\"\!\@\#\$\%\^\&\*\'\(\)\+\~\[\]\{\}\;\:\`\~\-\/\\\|\,\.\_]''' 
def punc_remove(x):
    b=[]
    for i in x:
        g=[]
        for _ in i:
            word = re.sub(symbollist,'',_)
            if len(word)>0:
                g.append(word)
        b.append(g)
    return b
df['New']=punc_remove(df['Lemmatize'])

In [16]:
#Export DF to csv
df.to_csv('clean_v2.csv')